In [1]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=1986e9f5c3f62e7a1ae6852d8d2539a2b7928cf0683b7788fada57879863b3e4
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
!pip install osgeo simplekml

In [ ]:
import googlemaps

GMAPS_API_KEY = ''
gmaps = googlemaps.Client(key=GMAPS_API_KEY)
def convert_xy_to_latlong(xy_coords, geotransform):
    """Convert pixel coordinates to geographic coordinates"""
    return [
        (geotransform[0] + x * geotransform[1] + y * geotransform[2],
         geotransform[3] + x * geotransform[4] + y * geotransform[5])
        for x, y in xy_coords
    ]

# Example usage:
geotransform = (long_origin, x_pixel_size, x_rotation, lat_origin, y_rotation, y_pixel_size)
destructed_polygons = convert_xy_to_latlong(xy_coords, geotransform)


In [ ]:
from osgeo import gdal, osr

def pixel_to_geo(pixel_x, pixel_y, geotransform):
    lon = geotransform[0] + pixel_x * geotransform[1] + pixel_y * geotransform[2]
    lat = geotransform[3] + pixel_x * geotransform[4] + pixel_y * geotransform[5]
    return lon, lat

def georeference_destructed_areas(destructed_areas, geotiff_path):
    ds = gdal.Open(geotiff_path)
    geotransform = ds.GetGeoTransform()
    
    geo_destructed_areas = []
    rows, cols = destructed_areas.shape
    for y in range(rows):
        for x in range(cols):
            if destructed_areas[y, x] == 1:
                lon, lat = pixel_to_geo(x, y, geotransform)
                geo_destructed_areas.append((lon, lat))
    
    return geo_destructed_areas

# Usage
geo_destructed_areas = georeference_destructed_areas(destructed_areas, 'georeferenced_satellite_image.tif')


In [ ]:
def create_destruction_map(polygons):
    poly_coords = ",\n".join(
        f"new google.maps.LatLng({lat}, {lng})" 
        for lng, lat in polygons[0]  # First polygon
    )
    
    return f"""
    <html>
    <head>
        <script src="https://maps.googleapis.com/maps/api/js?key={GMAPS_API_KEY}"></script>
        <style>#map {{ height: 600px; width: 100%; }}</style>
    </head>
    <body>
        <div id="map"></div>
        <script>
            function initMap() {{
                var map = new google.maps.Map(document.getElementById('map'), {{
                    zoom: 12,
                    center: new google.maps.LatLng({polygons[0][0][1]}, {polygons[0][0][0]})
                }});
                
                new google.maps.Polygon({{
                    paths: [{poly_coords}],
                    strokeColor: "#FF0000",
                    strokeOpacity: 0.8,
                    strokeWeight: 2,
                    fillColor: "#FF0000",
                    fillOpacity: 0.35,
                    map: map
                }});
            }}
            google.maps.event.addDomListener(window, 'load', initMap);
        </script>
    </body>
    </html>
    """

# Save and display
with open("destruction_map.html", "w") as f:
    f.write(create_destruction_map(destructed_polygons))
